In [15]:
import os, sys
import numpy as np
from matplotlib import pyplot as plt
from collections import defaultdict
import cv2
import json
from cloudvolume import CloudVolume
from taskqueue import LocalTaskQueue
import igneous.task_creation as tc
import shutil

In [2]:
animal = 'DK46'
HOME = os.path.expanduser("~")
PATH = os.path.join(HOME, 'programming/pipeline_utility/src')
sys.path.append(PATH)
from lib.sqlcontroller import SqlController
from lib.file_location import FileLocationManager
from lib.utilities_cvat_neuroglancer import NumpyToNeuroglancer, calculate_chunks
fileLocationManager = FileLocationManager(animal)
from lib.utilities_process import SCALING_FACTOR
THUMBNAIL_PATH = os.path.join(fileLocationManager.prep, 'CH1', 'thumbnail_aligned')
THUMBNAILS = sorted(os.listdir(THUMBNAIL_PATH))
num_sections = len(THUMBNAILS)
sqlController = SqlController(animal)
width = sqlController.scan_run.width
height = sqlController.scan_run.height
scale_xy = sqlController.scan_run.resolution
z_scale = sqlController.scan_run.zresolution
scales = np.array([scale_xy, scale_xy, z_scale])
width = int(width * SCALING_FACTOR)
height = int(height * SCALING_FACTOR)
aligned_shape = np.array((width, height))

In [3]:
rows = sqlController.get_annotations('DK46', 1, 'cerebellum')
polygons = defaultdict(list)

for row in rows:
    xy = (row.x/scale_xy, row.y/scale_xy)
    z = int(np.round(row.z/z_scale))
    polygons[z].append(xy)

In [4]:
volume = np.zeros((aligned_shape[1], aligned_shape[0], num_sections), dtype=np.uint8)
color = 100
for section,points in polygons.items():
    template = np.zeros((aligned_shape[1], aligned_shape[0]), dtype=np.uint8)
    points = np.array(points)
    points = np.round(points*SCALING_FACTOR)
    points = points.astype(np.int32)
    cv2.polylines(template, [points], True, color, 8, lineType=cv2.LINE_AA)
    volume[:, :, section - 1] = template

In [25]:
# Voxel resolution in nanometer (how much nanometer each element in numpy array represent)
scales = (10400, 10400, 20000)
# Voxel offset
offset = (0, 0, 0)
data_type = str(volume.dtype)
layer_type = 'segmentation'
chunks = [64, 64, 64]
num_channels = 1
downsample = True
volume_size = (width, height, num_sections) # neuroglancer is width, height

OUTPUT1_DIR = os.path.join(fileLocationManager.neuroglancer_data, 'mesh')

if os.path.exists(OUTPUT1_DIR):
    shutil.rmtree(OUTPUT1_DIR)

os.makedirs(OUTPUT1_DIR, exist_ok=True)


In [27]:
cloudpath = f'file://{OUTPUT1_DIR}'
info = CloudVolume.create_new_info(
    num_channels = num_channels,
    layer_type = layer_type,
    data_type = str(volume.dtype), # Channel images might be 'uint8'
    encoding = 'raw', # raw, jpeg, compressed_segmentation, fpzip, kempressed
    resolution = scales, # Voxel scaling, units are in nanometers
    voxel_offset = offset, # x,y,z offset in voxels from the origin
    chunk_size = [64, 64, 64], # units are voxels
    volume_size = volume.shape, # e.g. a cubic millimeter dataset
)
vol = CloudVolume(cloudpath, mip=0, info=info, compress=False)
vol.commit_info()
vol[:, :, :] = volume[:, :, :]

Uploading: 100%|██████████| 4560/4560 [00:32<00:00, 142.13it/s]


In [29]:
cv2 = CloudVolume(cloudpath, 0)
cv2.info['segment_properties'] = 'names'
cv2.commit_info()

segment_properties_path = os.path.join(cloudpath.replace('file://', ''), 'names')
os.makedirs(segment_properties_path, exist_ok=True)

info = {
    "@type": "neuroglancer_segment_properties",
    "inline": {
        "ids": [str(color)],
        "properties": [{
            "id": "label",
            "type": "label",
            "values": [str(color)]
        }]
    }
}
with open(os.path.join(segment_properties_path, 'info'), 'w') as file:
    json.dump(info, file, indent=2)


In [30]:
mse = 40
tq = LocalTaskQueue(parallel=False)
mesh_dir = f'mesh_mip_0_err_{mse}'
cv2.info['mesh'] = mesh_dir
cv2.commit_info()
tasks = tc.create_meshing_tasks(cv2.layer_cloudpath, mip=0, mesh_dir=mesh_dir, max_simplification_error=mse)
tq.insert(tasks)
tq.execute()


Tasks:   0%|          | 0/30 [00:00<?, ?it/s]

Downloading:   0%|          | 0/512 [00:00<?, ?it/s]

Downloading:   4%|▍         | 20/512 [00:00<00:03, 143.00it/s]

Downloading:   7%|▋         | 36/512 [00:00<00:02, 173.50it/s]

Downloading:   9%|▉         | 46/512 [00:00<00:02, 173.50it/s]

Downloading:  10%|█         | 52/512 [00:00<00:02, 173.50it/s]

Downloading:  12%|█▏        | 62/512 [00:00<00:02, 152.90it/s]

Downloading:  15%|█▌        | 77/512 [00:00<00:02, 152.90it/s]

Downloading:  18%|█▊        | 93/512 [00:00<00:02, 152.90it/s]

Downloading:  21%|██        | 106/512 [00:00<00:03, 115.69it/s]

Downloading:  25%|██▌       | 128/512 [00:00<00:06, 61.58it/s] 

Downloading:  27%|██▋       | 138/512 [00:00<00:06, 61.58it/s]

Downloading:  29%|██▉       | 148/512 [00:00<00:07, 46.11it/s]

Downloading:  29%|██▉       | 149/512 [00:00<00:07, 46.11it/s]

Downloading:  34%|███▍      | 175/512 [00:00<00:08, 39.01it/s]

Downloading:  36%|███▌      | 182/512 [00:00<00:08, 38.39it/s]

D

Downloading: 100%|██████████| 512/512 [00:02<00:00,  5.93it/s]

Tasks:   3%|▎         | 1/30 [00:02<01:04,  2.23s/it]

Downloading:   0%|          | 0/512 [00:00<?, ?it/s]

Downloading:  10%|█         | 52/512 [00:00<00:00, 514.05it/s]

Downloading:  10%|█         | 53/512 [00:00<00:00, 506.40it/s]

Downloading:  21%|██        | 106/512 [00:00<00:00, 467.50it/s]

Downloading:  21%|██        | 108/512 [00:00<00:00, 464.99it/s]

Downloading:  23%|██▎       | 116/512 [00:00<00:00, 464.99it/s]

Downloading:  23%|██▎       | 120/512 [00:00<00:00, 464.99it/s]

Downloading:  25%|██▌       | 128/512 [00:00<00:00, 464.99it/s]

Downloading:  39%|███▉      | 202/512 [00:00<00:01, 205.71it/s]

Downloading:  42%|████▏     | 217/512 [00:00<00:01, 192.84it/s]

Downloading:  45%|████▍     | 228/512 [00:00<00:01, 192.84it/s]

Downloading:  45%|████▌     | 232/512 [00:00<00:01, 192.84it/s]

Downloading:  49%|████▉     | 251/512 [00:00<00:01, 192.84it/s]

Downloading:  51%|█████     | 260/512 [00:00<00:0

Downloading:  83%|████████▎ | 427/512 [00:01<00:03, 26.58it/s]

Downloading:  85%|████████▌ | 436/512 [00:01<00:03, 23.59it/s]

Downloading:  87%|████████▋ | 445/512 [00:01<00:03, 20.20it/s]

Downloading:  88%|████████▊ | 451/512 [00:01<00:02, 21.03it/s]

Downloading:  89%|████████▉ | 456/512 [00:01<00:02, 20.56it/s]

Downloading:  90%|████████▉ | 460/512 [00:01<00:02, 19.00it/s]

Downloading:  91%|█████████ | 464/512 [00:01<00:02, 18.70it/s]

Downloading:  91%|█████████▏| 468/512 [00:01<00:02, 18.31it/s]

Downloading:  92%|█████████▏| 472/512 [00:01<00:02, 18.30it/s]

Downloading:  93%|█████████▎| 476/512 [00:01<00:01, 18.11it/s]

Downloading:  94%|█████████▍| 480/512 [00:01<00:01, 18.35it/s]

Downloading:  95%|█████████▍| 484/512 [00:01<00:01, 18.40it/s]

Downloading:  95%|█████████▌| 487/512 [00:01<00:01, 17.47it/s]

Downloading:  96%|█████████▌| 490/512 [00:01<00:01, 17.64it/s]

Downloading:  96%|█████████▋| 493/512 [00:01<00:01, 17.65it/s]

Downloading:  97%|█████████▋| 496/512 [0

Downloading: 100%|██████████| 320/320 [00:00<00:00, 82.31it/s]

Downloading: 100%|██████████| 320/320 [00:00<00:00, 82.31it/s]

Downloading: 100%|██████████| 320/320 [00:00<00:00, 82.31it/s]

Tasks:  20%|██        | 6/30 [00:09<00:32,  1.36s/it]

Downloading:   0%|          | 0/512 [00:00<?, ?it/s]

Downloading:   8%|▊         | 43/512 [00:00<00:01, 314.88it/s]

Downloading:  10%|▉         | 50/512 [00:00<00:01, 319.74it/s]

Downloading:  13%|█▎        | 67/512 [00:00<00:01, 319.74it/s]

Downloading:  15%|█▍        | 75/512 [00:00<00:01, 319.74it/s]

Downloading:  19%|█▉        | 99/512 [00:00<00:01, 319.74it/s]

Downloading:  24%|██▎       | 121/512 [00:00<00:01, 319.74it/s]

Downloading:  28%|██▊       | 144/512 [00:00<00:02, 145.84it/s]

Downloading:  34%|███▎      | 172/512 [00:00<00:02, 145.84it/s]

Downloading:  38%|███▊      | 196/512 [00:00<00:03, 82.53it/s] 

Downloading:  40%|████      | 205/512 [00:00<00:03, 82.53it/s]

Downloading:  41%|████      | 210/512 [00:00<00:03, 82.

Downloading: 100%|██████████| 512/512 [00:01<00:00, 20.67it/s]

Downloading: 100%|██████████| 512/512 [00:01<00:00, 20.67it/s]

Downloading: 100%|██████████| 512/512 [00:01<00:00, 20.67it/s]

Tasks:  27%|██▋       | 8/30 [00:52<04:36, 12.57s/it]

Downloading:   0%|          | 0/320 [00:00<?, ?it/s]

Downloading:   6%|▌         | 19/320 [00:00<00:01, 161.10it/s]

Downloading:  10%|█         | 33/320 [00:00<00:01, 238.08it/s]

Downloading:  12%|█▏        | 39/320 [00:00<00:01, 238.08it/s]

Downloading:  15%|█▍        | 47/320 [00:00<00:01, 238.08it/s]

Downloading:  16%|█▋        | 52/320 [00:00<00:01, 238.08it/s]

Downloading:  21%|██        | 67/320 [00:00<00:01, 238.08it/s]

Downloading:  24%|██▍       | 78/320 [00:00<00:01, 238.08it/s]

Downloading:  30%|██▉       | 95/320 [00:00<00:01, 175.75it/s]

Downloading:  34%|███▍      | 109/320 [00:00<00:01, 175.75it/s]

Downloading:  37%|███▋      | 118/320 [00:00<00:01, 175.75it/s]

Downloading:  43%|████▎     | 137/320 [00:00<00:02, 75.62

Downloading:  96%|█████████▌| 490/512 [00:01<00:03,  7.30it/s]

Downloading:  96%|█████████▌| 490/512 [00:01<00:03,  7.30it/s]

Downloading:  96%|█████████▌| 492/512 [00:01<00:02,  7.40it/s]

Downloading:  96%|█████████▌| 492/512 [00:01<00:02,  7.40it/s]

Downloading:  96%|█████████▋| 493/512 [00:01<00:02,  6.78it/s]

Downloading:  96%|█████████▋| 494/512 [00:01<00:02,  6.36it/s]

Downloading:  97%|█████████▋| 495/512 [00:01<00:02,  6.11it/s]

Downloading:  97%|█████████▋| 497/512 [00:01<00:01,  7.72it/s]

Downloading:  97%|█████████▋| 498/512 [00:01<00:01,  7.29it/s]

Downloading:  97%|█████████▋| 499/512 [00:01<00:01,  7.02it/s]

Downloading:  97%|█████████▋| 499/512 [00:01<00:01,  7.02it/s]

Downloading:  98%|█████████▊| 501/512 [00:01<00:01,  7.33it/s]

Downloading:  98%|█████████▊| 503/512 [00:01<00:00,  9.26it/s]

Downloading:  98%|█████████▊| 504/512 [00:01<00:00,  8.84it/s]

Downloading:  99%|█████████▉| 506/512 [00:01<00:00, 10.42it/s]

Downloading:  99%|█████████▉| 507/512 [0

Downloading:  79%|███████▉  | 406/512 [00:01<00:17,  6.13it/s]

Downloading:  80%|███████▉  | 408/512 [00:01<00:15,  6.59it/s]

Downloading:  80%|███████▉  | 409/512 [00:01<00:16,  6.21it/s]

Downloading:  80%|████████  | 410/512 [00:01<00:16,  6.21it/s]

Downloading:  80%|████████  | 411/512 [00:01<00:16,  6.21it/s]

Downloading:  81%|████████  | 414/512 [00:01<00:15,  6.46it/s]

Downloading:  81%|████████  | 414/512 [00:01<00:15,  6.46it/s]

Downloading:  81%|████████▏ | 416/512 [00:01<00:16,  5.66it/s]

Downloading:  81%|████████▏ | 417/512 [00:01<00:17,  5.40it/s]

Downloading:  82%|████████▏ | 418/512 [00:01<00:17,  5.40it/s]

Downloading:  82%|████████▏ | 422/512 [00:01<00:13,  6.45it/s]

Downloading:  83%|████████▎ | 423/512 [00:01<00:15,  5.89it/s]

Downloading:  83%|████████▎ | 424/512 [00:01<00:15,  5.51it/s]

Downloading:  83%|████████▎ | 425/512 [00:01<00:16,  5.28it/s]

Downloading:  83%|████████▎ | 426/512 [00:01<00:16,  5.28it/s]

Downloading:  84%|████████▎ | 428/512 [0

Downloading:  59%|█████▉    | 190/320 [00:00<00:01, 100.30it/s]

Downloading:  68%|██████▊   | 216/320 [00:00<00:01, 100.30it/s]

Downloading:  74%|███████▍  | 238/320 [00:00<00:00, 100.30it/s]

Downloading:  83%|████████▎ | 265/320 [00:00<00:00, 100.30it/s]

Downloading:  91%|█████████ | 291/320 [00:00<00:00, 100.30it/s]

Downloading:  99%|█████████▉| 316/320 [00:00<00:00, 59.14it/s] 

Downloading: 100%|██████████| 320/320 [00:00<00:00, 59.14it/s]

Downloading: 100%|██████████| 320/320 [00:00<00:00, 59.14it/s]

Downloading: 100%|██████████| 320/320 [00:00<00:00, 59.14it/s]

Downloading: 100%|██████████| 320/320 [00:00<00:00, 59.14it/s]

Downloading: 100%|██████████| 320/320 [00:00<00:00, 59.14it/s]

Downloading: 100%|██████████| 320/320 [00:00<00:00, 59.14it/s]

Downloading: 100%|██████████| 320/320 [00:00<00:00, 59.14it/s]

Downloading: 100%|██████████| 320/320 [00:01<00:00, 59.14it/s]

Tasks:  40%|████      | 12/30 [01:24<02:38,  8.81s/it]

Downloading:   0%|          | 0/128 [00:00

Downloading:  75%|███████▌  | 48/64 [00:00<00:00, 453.69it/s]

Downloading:  75%|███████▌  | 48/64 [00:00<00:00, 453.69it/s]

Tasks:  73%|███████▎  | 22/30 [01:27<00:03,  2.04it/s]

Downloading:   0%|          | 0/64 [00:00<?, ?it/s]

Downloading:  91%|█████████ | 58/64 [00:00<00:00, 528.25it/s]

Downloading: 100%|██████████| 64/64 [00:00<00:00, 531.65it/s]

Downloading: 100%|██████████| 64/64 [00:00<00:00, 531.65it/s]

Downloading: 100%|██████████| 64/64 [00:00<00:00, 531.65it/s]

Downloading: 100%|██████████| 64/64 [00:00<00:00, 531.65it/s]

Downloading: 100%|██████████| 64/64 [00:00<00:00, 531.65it/s]

Downloading: 100%|██████████| 64/64 [00:00<00:00, 531.65it/s]

Downloading: 100%|██████████| 64/64 [00:00<00:00, 531.65it/s]

Tasks:  77%|███████▋  | 23/30 [01:27<00:02,  2.52it/s]

Tasks:  80%|████████  | 24/30 [01:27<00:01,  3.03it/s]

Downloading:   0%|          | 0/64 [00:00<?, ?it/s]

Downloading:  75%|███████▌  | 48/64 [00:00<00:00, 431.00it/s]

Downloading:  88%|████████▊ | 56/

In [31]:
##### 2nd mesh task, create manifest
tasks = tc.create_mesh_manifest_tasks(cv2.layer_cloudpath, mesh_dir=mesh_dir)
tq.insert(tasks)
tq.execute()


Tasks: 100%|██████████| 999/999 [00:33<00:00, 30.24it/s]
